Henter innsamlett data



oversikt over forskjellig type skrift

# OVERSKRIFT størst

## Overskrift mindre

### Overskrift minst


Fet	**tekst**	fet tekst

Kursiv	*tekst*	kursiv tekst

Begge	***tekst***	fet og kursiv